In [1]:
import pandas as pd

utrecht = '/home/wouter/Documents/Scriptie/utrecht_addresses.csv'
utrecht_df = pd.read_csv(utrecht)
display(utrecht_df.head())

postcode huisnummer               straat  \
0   3511KZ          1     3e Buurkerksteeg   
1   3511JH          1    Achter Clarenburg   
2   3511MR          1  Albert Verweystraat   
3   3511LM          1       Alendorpstraat   
4   3511LM       1-BS       Alendorpstraat   

                                            buurt                wijk  \
0  Lange Elisabethstraat, Mariaplaats en omgeving  Wijk 06 Binnenstad   
1  Lange Elisabethstraat, Mariaplaats en omgeving  Wijk 06 Binnenstad   
2                                  Hooch Boulandt  Wijk 06 Binnenstad   
3  Lange Elisabethstraat, Mariaplaats en omgeving  Wijk 06 Binnenstad   
4  Lange Elisabethstraat, Mariaplaats en omgeving  Wijk 06 Binnenstad   

  woonplaats gemeente provincie   latitude  longitude  
0    Utrecht  Utrecht   Utrecht  52.090449   5.118523  
1    Utrecht  Utrecht   Utrecht  52.090584   5.116154  
2    Utrecht  Utrecht   Utrecht  52.082053   5.120199  
3    Utrecht  Utrecht   Utrecht  52.089463   5.118824  
4    Utrecht  Utrecht   Utrecht  52.089475   5.118816

In [2]:
print(len(utrecht_df))

175299


In [3]:
def initialize_csv(file_path: str):
    # Define the columns as per the specified labels
    columns = [
        "postcode", "huisnummer", "straat", "buurt", "bouwjaar", "grondoppervlakte", "oppervlakte",
        "woz_2014", "woz_2015", "woz_2016", "woz_2017", "woz_2018",
        "woz_2019", "woz_2020", "woz_2021", "woz_2022", "woz_2023"
        "latitude", "longitude"
    ]

    # Create an empty DataFrame with these columns
    df = pd.DataFrame(columns=columns)

    # Save to CSV
    df.to_csv(file_path, index=False)
    print(f"Initialized CSV at: {file_path}")

file_path = 'merged_utrecht.csv'
initialize_csv(file_path)

Initialized CSV at: merged_utrecht.csv


In [5]:
import pandas as pd
import re

def initialize_csv(file_path: str):
    columns = [
        "postcode", "huisnummer", "straat", "buurt", "bouwjaar", "grondoppervlakte", "oppervlakte",
        "woz_2014", "woz_2015", "woz_2016", "woz_2017", "woz_2018",
        "woz_2019", "woz_2020", "woz_2021", "woz_2022", "woz_2023",
        "latitude", "longitude"
    ]
    
    df = pd.DataFrame(columns=columns)
    df.to_csv(file_path, index=False)
    print(f"Initialized CSV at: {file_path}")


def extract_street_and_number(address):
    match = re.match(r"(.*?)(\d+.*)", address)
    if match:
        street = match.group(1).strip()
        number = match.group(2).lstrip("0").strip()
        return street, number
    return address.strip(), ""



def merge_files(file_path, utrecht_path):
    initialize_csv(file_path)
    
    merged_df = pd.read_csv(file_path)
    utrecht_df = pd.read_csv(utrecht_path)


    # Define the desired data types for the final DataFrame
    dtypes = {
        "postcode": "string",
        "huisnummer": "string",
        "straat": "string",
        "buurt": "string",
        "bouwjaar": "Int64",  # Use Int64 for nullable integers
        "grondoppervlakte": "Int64",
        "oppervlakte": "Int64",
        "woz_2014": "Int64",
        "woz_2015": "Int64",
        "woz_2016": "Int64",
        "woz_2017": "Int64",
        "woz_2018": "Int64",
        "woz_2019": "Int64",
        "woz_2020": "Int64",
        "woz_2021": "Int64",
        "woz_2022": "Int64",
        "woz_2023": "Int64",
        "latitude": "float",
        "longitude": "float"
    }
    
    for nr in range(1, 15):  # Adjust range as needed
        file = f'{nr}.csv'
        print(f"Processing: {file}")
        
        try:
            prop_df = pd.read_csv(file)
            prop_df = prop_df

            prop_df["straat"], prop_df["huisnummer"] = zip(*prop_df["address"].apply(extract_street_and_number))
            prop_df.drop(columns=["address"], inplace=True)  
            

            # Merge dataframes on columns 'A' and 'B' (default is inner join)
            merged_prop = pd.merge(
                prop_df[["huisnummer", "straat", "bouwjaar", "grondoppervlakte", "oppervlakte",
                        "woz_2014", "woz_2015", "woz_2016", "woz_2017", "woz_2018",
                        "woz_2019", "woz_2020", "woz_2021", "woz_2022", "woz_2023"]],
                utrecht_df[["postcode", "huisnummer", "straat", "buurt", "latitude", "longitude"]], 
                on=['huisnummer', 'straat'], how='inner')
            #print(merged_prop)
            
            # Select the final columns for merged_df
            final_columns = [
                "postcode", "huisnummer", "straat", "buurt", "bouwjaar", "grondoppervlakte", "oppervlakte",
                "woz_2014", "woz_2015", "woz_2016", "woz_2017", "woz_2018",
                "woz_2019", "woz_2020", "woz_2021", "woz_2022", "woz_2023",
                "latitude", "longitude"
            ]

            # Create the final merged DataFrame with the selected columns
            merged_prop = merged_prop[final_columns]

            
            # Replace NaN with pd.NA
            merged_prop = merged_prop.fillna(pd.NA)
            
            # Ensure the data types are consistent
            merged_prop = merged_prop.astype(dtypes)

            # Append to the merged DataFrame
            merged_df = pd.concat([merged_df, merged_prop], ignore_index=True)

            # Debugging output to check the merged DataFrame
            #print("Merged DataFrame:")
            #print(merged_df)

        except Exception as e:
            print(f"Could not open {file}: {e}")
            continue

    # Save the merged DataFrame to CSV
    merged_df.to_csv(file_path, index=False)
    print(f"Merged data saved to: {file_path}")


file_path = 'merged_utrecht.csv'
utrecht_path = '/home/wouter/Documents/Scriptie/utrecht_addresses.csv'

# Initialize and merge\initialize_csv(file_path)
merge_files(file_path, utrecht_path)


Initialized CSV at: merged_utrecht.csv
Processing: 1.csv
Processing: 2.csv


/tmp/ipykernel_53853/3815392474.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([merged_df, merged_prop], ignore_index=True)


Processing: 3.csv
Processing: 4.csv
Processing: 5.csv
Processing: 6.csv
Processing: 7.csv
Processing: 8.csv
Processing: 9.csv
Processing: 10.csv
Processing: 11.csv
Processing: 12.csv
Processing: 13.csv
Processing: 14.csv
Merged data saved to: merged_utrecht.csv


In [6]:
def finalize_csv_format(file_path: str):
    """
    Finalize the CSV file by grouping rows and sorting them by gm_naam, gwb_code, and year.
    """
    # List of buurten (neighborhoods)
    buurten = [
        'Lange Elisabethstraat, Mariaplaats en omgeving', 'Hooch Boulandt',
        'Springweg en omgeving Geertebuurt', 'Wijk C',
        'Hoog-Catharijne NS en Jaarbeurs', 'Domplein, Neude, Janskerkhof',
        'Bleekstraat en omgeving', 'Lange Nieuwstraat en omgeving',
        'Breedstraat en Plompetorengracht en omgeving', 'Nobelstraat en omgeving',
        'Nieuwegracht-Oost', 'Pijlsweerd-Zuid', 'Pijlsweerd-Noord', 'Rivierenwijk',
        'Vogelenbuurt', 'Lauwerecht', 'Tuinwijk-Oost', 'Staatsliedenbuurt',
        'Tuinwijk-West', 'Dichterswijk', 'Tolsteeg en Rotsoord',
        'Oud Hoograven-Noord', 'Bokkenbuurt', 'Nieuw Hoograven-Noord',
        'Lunetten-Zuid', 'Lunetten-Noord', 'Oud Hoograven-Zuid',
        'Nieuw Hoograven-Zuid', 'Kanaleneiland-Zuid', 'Transwijk-Zuid',
        'Bedrijvengebied Kanaleneiland', 'Kanaleneiland-Noord', 'Transwijk-Noord',
        'Welgelegen, Den Hommel', 'Bedrijvengebied Papendorp', 'Leeuwesteyn',
        'Lombok-West', 'Laan van Nieuw-Guinea, Spinozaweg e.o.', 'Lombok-Oost',
        'Leidseweg en omgeving',
        'Nieuw Engeland, Th. a. Kempisplantsoen en omgeving', 'Halve Maan-Zuid',
        'Oog in Al', 'Halve Maan-Noord', 'Schepenbuurt, Cartesiusweg e.o.',
        'Grauwaart', 'Leidsche Rijn-Centrum', 'Hoge Weide', 'Langerak',
        'Bedrijventerrein Lageweide', 'Terwijde-Oost', 'Terwijde-West', 'Haarrijn',
        'Bedrijventerrein De Wetering', 'Het Zand-West', 'Parkwijk-Zuid',
        'Het Zand-Oost', 'Parkwijk-Noord', 'Rijnvliet', 'Rijnenburg',
        'Bedrijvengebied Strijkviertel', '2e Daalsebuurt en omgeving',
        'Egelantierstraat, Mariëndaalstraat e.o.',
        'Nijenoord, Hoogstraat en omgeving', 'Julianapark en omgeving', 'Ondiep',
        'Zamenhofdreef en omgeving', 'Geuzenwijk', 'Vechtzoom-zuid',
        'Elinkwijk en omgeving', 'Schaakbuurt en omgeving',
        'Prins Bernhardplein en omgeving', 'Queeckhovenplein en omgeving',
        'Zuilen-Noord', 'Wolga- en Donaudreef en omgeving',
        'Taag- en Rubicondreef en omgeving', 'Neckardreef en omgeving',
        'Poldergebied Overvecht', 'Vechtzoom-noord, Klopvaart',
        'Zambesidreef en omgeving', 'Tigrisdreef en omgeving',
        'Bedrijventerrein en omgeving', 'Voordorp en Voorveldsepolder',
        'Tuindorp en Van Lieflandlaan-West', 'Tuindorp-Oost',
        'Zeeheldenbuurt, Hengeveldstraat en omgeving', 'Buiten Wittevrouwen',
        'Wittevrouwen', 'Huizingalaan, K. Doormanlaan en omgeving', 'Oudwijk',
        'Wilhelminapark en omgeving', 'Abstede, Tolsteegsingel e.o.',
        'Watervogelbuurt', 'Rubenslaan en omgeving',
        'L. Napoleonplantsoen en omgeving', 'Sterrenwijk', 'Schildersbuurt',
        'Rijnsweerd', 'Utrecht Science Park', 'Galgenwaard en omgeving',
        'Maarschalkerweerd en Mereveld'
        ]
    
    try:
        # Load the CSV
        df = pd.read_csv(file_path)
        
        # Check which buurten are present in the DataFrame
        buurten_in_df = df['buurt'].unique()
        
        # Find missing buurten
        missing_buurten = set(buurten) - set(buurten_in_df)
        
        # Print missing buurten
        if missing_buurten:
            print("The following buurten are missing from the DataFrame:")
            print(missing_buurten)
        else:
            print("All buurten are present in the DataFrame.")
        
        # Drop duplicate rows
        df = df.drop_duplicates()


        # Sort the DataFrame by gm_naam, gwb_code, and year
        df = df.sort_values(by=["buurt", "straat", "huisnummer"], ascending=[True, True, True])

        # Save back to the same CSV 
        df.to_csv(file_path, index=False)
        print(f"CSV file '{file_path}' has been formatted and saved.")
    except Exception as e:
        print(f"Error during finalizing CSV: {e}")


finalize_csv_format('merged_utrecht.csv')


The following buurten are missing from the DataFrame:
{'Bedrijvengebied Strijkviertel', 'Bedrijventerrein De Wetering', 'Haarrijn'}
CSV file 'merged_utrecht.csv' has been formatted and saved.
